<a href="https://colab.research.google.com/github/ronakdm/input-marginalization/blob/main/lstm_snli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ronakdm/input-marginalization.git

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%bash
cd input-marginalization
git pull
cd ..

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
save_dir = "/content/gdrive/My Drive/input-marginalization"

In [2]:
import pickle
import numpy as np
import time
import datetime
import random
import torch
import torch.nn as nn

In [3]:
#import sys
#sys.path.append("input-marginalization")

from utils import generate_snli_dataloader, train, test
from models import SNLILSTM

In [4]:
try:
    from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup, BertTokenizer
except ModuleNotFoundError:
    !pip install transformers
    from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup, BertTokenizer
    

In [5]:
LEARNING_RATE = 1e-4
ADAMW_TOLERANCE = 1e-8
BATCH_SIZE = 128
EPOCHS = 30
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
VOCAB_SIZE = tokenizer.vocab_size
LABEL_SIZE = 3
HIDDEN_DIM = 200
EMBEDDING_DIM = 100
N_RNN_LAYERS = 2

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

device='cpu'
print("Running on '%s'." % device)

Running on 'cpu'.


In [6]:
pre = 'preprocessed_data/SNLI/snli_1.0/'
train_dataloader, validation_dataloader, test_dataloader = generate_snli_dataloader(pre, BATCH_SIZE)

In [7]:
model = SNLILSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, LABEL_SIZE, N_RNN_LAYERS).to(device)
save_filename = "lstm_snli"
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps = ADAMW_TOLERANCE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = EPOCHS * BATCH_SIZE * len(train_dataloader))

In [ ]:
save_dir='.'
try:
    train(model, EPOCHS, train_dataloader, validation_dataloader, optimizer, scheduler, save_dir, save_filename, device, dataset='snli')
except KeyboardInterrupt:
    print("Graceful Exit")